# REVENUE BY MONTH YEAR

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///olist.db")
# Paso 1: Query final con columna month y los 12 meses garantizados
query_final = """
SELECT
    strftime('%m', o.order_delivered_customer_date) AS month_no,
    CASE strftime('%m', o.order_delivered_customer_date)
        WHEN '01' THEN 'Jan'
        WHEN '02' THEN 'Feb'
        WHEN '03' THEN 'Mar'
        WHEN '04' THEN 'Apr'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'Jun'
        WHEN '07' THEN 'Jul'
        WHEN '08' THEN 'Aug'
        WHEN '09' THEN 'Sep'
        WHEN '10' THEN 'Oct'
        WHEN '11' THEN 'Nov'
        WHEN '12' THEN 'Dec'
    END AS month,
    ROUND(SUM(CASE WHEN strftime('%Y', o.order_delivered_customer_date) = '2016' THEN p.payment_value ELSE 0 END), 2) AS Year2016,
    ROUND(SUM(CASE WHEN strftime('%Y', o.order_delivered_customer_date) = '2017' THEN p.payment_value ELSE 0 END), 2) AS Year2017,
    ROUND(SUM(CASE WHEN strftime('%Y', o.order_delivered_customer_date) = '2018' THEN p.payment_value ELSE 0 END), 2) AS Year2018
FROM olist_orders AS o

JOIN (
    SELECT order_id, MIN(payment_value) AS payment_value
    FROM olist_order_payments
    GROUP BY order_id
) AS p
ON o.order_id = p.order_id
WHERE o.order_status = 'delivered'
  AND o.order_delivered_customer_date IS NOT NULL
GROUP BY month_no, month
ORDER BY month_no;
"""

df_final = pd.read_sql_query(text(query_final), engine)
print("Query FINAL - Revenue por mes y año:")
print(df_final)

Query FINAL - Revenue por mes y año:
   month_no month  Year2016    Year2017    Year2018
0        01   Jan      0.00    37632.57   969967.80
1        02   Feb      0.00   222270.75   853616.82
2        03   Mar      0.00   376833.72  1024851.95
3        04   Apr      0.00   299798.45  1274742.18
4        05   May      0.00   579280.43  1150528.93
5        06   Jun      0.00   489463.42  1141543.85
6        07   Jul      0.00   518115.19   925958.79
7        08   Aug      0.00   609180.34  1319737.66
8        09   Sep      0.00   652576.48    12875.18
9        10   Oct  34116.28   740570.40      347.95
10       11   Nov  10734.64   733047.33        0.00
11       12   Dec    960.85  1082600.69        0.00


# top_10_revenue_categories

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///olist.db")
query_final1 = """
SELECT
    pcn.product_category_name_english AS Category,
    COUNT(DISTINCT o.order_id) AS Num_order,
    ROUND(SUM(op.payment_value), 2) AS Revenue
FROM olist_orders o
JOIN olist_order_items oi USING(order_id)
JOIN olist_products pr USING(product_id)
JOIN product_category_name_translation pcn USING(product_category_name)
JOIN olist_order_payments op USING(order_id)
WHERE 
    o.order_status = 'delivered'
    AND o.order_delivered_customer_date IS NOT NULL
    AND pcn.product_category_name_english IS NOT NULL
GROUP BY pcn.product_category_name_english
ORDER BY Revenue DESC
LIMIT 10;
"""

df_final1 = pd.read_sql_query(text(query_final1), engine)
print("Query FINAL - Top 10 Revenue Categories:")
print(df_final1)

Query FINAL - Top 10 Revenue Categories:
                Category  Num_order     Revenue
0         bed_bath_table       9272  1692714.28
1          health_beauty       8646  1620684.04
2  computers_accessories       6529  1549252.47
3        furniture_decor       6307  1394466.93
4          watches_gifts       5493  1387046.31
5         sports_leisure       7529  1349252.93
6             housewares       5743  1069787.97
7                   auto       3809   833610.84
8           garden_tools       3448   810614.93
9             cool_stuff       3559   744649.32


# top_10_least_revenue_categories

In [36]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///olist.db")
query_final2 = """
SELECT
    pcn.product_category_name_english AS Category,
    COUNT(DISTINCT o.order_id) AS Num_order,
    ROUND(SUM(op.payment_value), 2) AS Revenue
FROM olist_orders o
JOIN olist_order_items oi USING(order_id)
JOIN olist_products pr USING(product_id)
JOIN product_category_name_translation pcn USING(product_category_name)
JOIN olist_order_payments op USING(order_id)
WHERE 
    o.order_status = 'delivered'
    AND o.order_delivered_customer_date IS NOT NULL
    AND pcn.product_category_name_english IS NOT NULL
GROUP BY pcn.product_category_name_english
ORDER BY Revenue ASC
LIMIT 10;
"""

df_final2 = pd.read_sql_query(text(query_final2), engine)
print("Query FINAL - Top 10 least Revenue Categories:")
print(df_final2)

Query FINAL - Top 10 least Revenue Categories:
                    Category  Num_order  Revenue
0      security_and_services          2   324.51
1  fashion_childrens_clothes          7   718.98
2          cds_dvds_musicals         12  1199.43
3             home_comfort_2         24  1710.54
4                    flowers         29  2213.01
5      arts_and_craftmanship         23  2326.17
6                 la_cuisine         13  2913.53
7              fashion_sport         26  3605.83
8        diapers_and_hygiene         25  4126.17
9     fashio_female_clothing         36  4902.67


# revenue_per_state

In [41]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///olist.db")
query_final3 = """
SELECT
    c.customer_state AS customer_state,
    ROUND(SUM(p.payment_value), 2) AS Revenue
FROM olist_orders AS o
JOIN olist_customers AS c
    USING(customer_id)
JOIN (
    SELECT order_id, SUM(payment_value) AS payment_value
    FROM olist_order_payments
    GROUP BY order_id
) AS p
    USING(order_id)
WHERE o.order_status = 'delivered'
  AND o.order_delivered_customer_date IS NOT NULL
GROUP BY c.customer_state
ORDER BY Revenue DESC
LIMIT 10;
"""

df_final3 = pd.read_sql_query(text(query_final3), engine)
print("Query FINAL - Revenue per state:")
print(df_final3)

Query FINAL - Revenue per state:
  customer_state     Revenue
0             SP  5769081.27
1             RJ  2055690.45
2             MG  1819277.61
3             RS   861608.40
4             PR   781919.55
5             SC   595208.40
6             BA   591270.60
7             DF   346146.17
8             GO   334294.22
9             ES   317682.65


# Delivert date difference 

In [58]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///olist.db")
query_final4 = """
SELECT
    c.customer_state AS State,
    CAST(
        AVG(
            julianday(strftime('%Y-%m-%d', o.order_estimated_delivery_date))
            - julianday(strftime('%Y-%m-%d', o.order_delivered_customer_date))
        ) AS INTEGER
    ) AS Delivery_Difference
FROM olist_orders AS o
JOIN olist_customers AS c
    USING(customer_id)
WHERE o.order_status = 'delivered'
  AND o.order_delivered_customer_date IS NOT NULL
  AND o.order_estimated_delivery_date IS NOT NULL
GROUP BY c.customer_state
ORDER BY Delivery_Difference ASC;
"""

df_final4 = pd.read_sql_query(text(query_final4), engine)
print("Query FINAL - Delivery Difference per state:")
print(df_final4)

Query FINAL - Delivery Difference per state:
   State  Delivery_Difference
0     AL                    8
1     MA                    9
2     BA                   10
3     CE                   10
4     ES                   10
5     SE                   10
6     MS                   11
7     PI                   11
8     RJ                   11
9     SC                   11
10    SP                   11
11    DF                   12
12    GO                   12
13    TO                   12
14    MG                   13
15    PB                   13
16    PE                   13
17    PR                   13
18    RN                   13
19    RS                   13
20    MT                   14
21    PA                   14
22    RR                   17
23    AM                   19
24    AP                   19
25    AC                   20
26    RO                   20


# real_vs_estimated_delivered_time

In [84]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///olist.db")
query_final5 = """
SELECT
    strftime('%m', o.order_purchase_timestamp) AS month_no,
    CASE strftime('%m', o.order_purchase_timestamp)
        WHEN '01' THEN 'Jan'
        WHEN '02' THEN 'Feb'
        WHEN '03' THEN 'Mar'
        WHEN '04' THEN 'Apr'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'Jun'
        WHEN '07' THEN 'Jul'
        WHEN '08' THEN 'Aug'
        WHEN '09' THEN 'Sep'
        WHEN '10' THEN 'Oct'
        WHEN '11' THEN 'Nov'
        WHEN '12' THEN 'Dec'
    END AS month,
    
    -- Real delivery time (days)
    AVG(CASE WHEN strftime('%Y', o.order_purchase_timestamp) = '2016' THEN
        julianday( o.order_delivered_customer_date) -
        julianday( o.order_purchase_timestamp)
    END) AS Year2016_real_time,

    AVG(CASE WHEN strftime('%Y', o.order_purchase_timestamp) = '2017' THEN
        julianday( o.order_delivered_customer_date) -
        julianday( o.order_purchase_timestamp)
    END) AS Year2017_real_time,

    AVG(CASE WHEN strftime('%Y', o.order_purchase_timestamp) = '2018' THEN
        julianday( o.order_delivered_customer_date) -
        julianday( o.order_purchase_timestamp)
    END) AS Year2018_real_time,

    -- Estimated delivery time (days)
    AVG(CASE WHEN strftime('%Y', o.order_purchase_timestamp) = '2016' THEN
        julianday( o.order_estimated_delivery_date) -
        julianday( o.order_purchase_timestamp)
    END) AS Year2016_estimated_time,

    AVG(CASE WHEN strftime('%Y', o.order_purchase_timestamp) = '2017' THEN
        julianday( o.order_estimated_delivery_date) -
        julianday( o.order_purchase_timestamp)
    END) AS Year2017_estimated_time,

    AVG(CASE WHEN strftime('%Y', o.order_purchase_timestamp) = '2018' THEN
        julianday( o.order_estimated_delivery_date) -
        julianday( o.order_purchase_timestamp)
    END) AS Year2018_estimated_time

FROM (
    SELECT DISTINCT order_id, order_status,
        order_purchase_timestamp,
        order_delivered_customer_date,
        order_estimated_delivery_date
    FROM olist_orders
) AS o
WHERE o.order_status = 'delivered'
  AND o.order_delivered_customer_date IS NOT NULL
GROUP BY month_no, month
ORDER BY month_no;

"""

df_final5 = pd.read_sql_query(text(query_final5), engine)
print("Query FINAL - real_vs_estimated_delivered_time:")
print(df_final5)

Query FINAL - real_vs_estimated_delivered_time:
   month_no month  Year2016_real_time  Year2017_real_time  Year2018_real_time  \
0        01   Jan                 NaN           12.647044           14.080563   
1        02   Feb                 NaN           13.168825           16.945138   
2        03   Mar                 NaN           12.951184           16.301410   
3        04   Apr                 NaN           14.917913           11.499241   
4        05   May                 NaN           11.322363           11.419029   
5        06   Jun                 NaN           12.011573            9.239216   
6        07   Jul                 NaN           11.592732            8.958636   
7        08   Aug                 NaN           11.147125            7.732780   
8        09   Sep           54.813194           11.851007                 NaN   
9        10   Oct           19.600559           11.856700                 NaN   
10       11   Nov                 NaN           15.161132    

# global_amount_order_status1

In [103]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///olist.db")
query_final6 = """
SELECT
    o.order_status AS order_status,
    COUNT(o.order_id) AS Amount
FROM olist_orders AS o
GROUP BY o.order_status
ORDER BY o.order_status ASC;
"""

df_final6 = pd.read_sql_query(text(query_final6), engine)
print("Query FINAL - global amount order status:")
print(df_final6)

Query FINAL - global amount order status:
  order_status  Amount
0     approved       2
1     canceled     625
2      created       5
3    delivered   96478
4     invoiced     314
5   processing     301
6      shipped    1107
7  unavailable     609
